# [Going Deeper NLP 14] 한국어 QA 모델 만들기
한국어 bAbI 데이터셋으로 memory network를 구현해 학습한다. 

---

## 프로젝트 목표
---
- customized konlpy를 사용한다.
- memory network를 이해하고 구현한다.  

## 프로젝트 설명
---
1. 단어 토큰화
    - customized konlpy를 사용해 사전을 만든다.
    
2. 불용어 처리 
3. memory network를 한국어 데이터로 성능 비교하기 

    
## 0. Import module, library

In [1]:
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import tarfile
from nltk import FreqDist
from functools import reduce
import os
import re

In [2]:
data_dir = os.getenv('HOME')+'/aiffel/babi_memory_net'
train_path = data_dir + '/qa1_single-supporting-fact_train_kor.txt'
test_path = data_dir + '/qa1_single-supporting-fact_test_kor.txt'

- 데이터 샘플 확인

In [3]:
i = 0
lines = open(train_path , "rb")
for line in lines:
    line = line.decode("utf-8").strip()
    # lno, text = line.split(" ", 1) # ID와 TEXT 분리
    i = i + 1
    print(line)
    if i == 20:
        break

1 필웅이는 화장실로 갔습니다.
2 은경이는 복도로 이동했습니다.
3 필웅이는 어디야? 	화장실	1
4 수종이는 복도로 복귀했습니다.
5 경임이는 정원으로 갔습니다.
6 수종이는 어디야? 	복도	4
7 은경이는 사무실로 갔습니다.
8 경임이는 화장실로 뛰어갔습니다.
9 수종이는 어디야? 	복도	4
10 필웅이는 복도로 갔습니다.
11 수종이는 사무실로 가버렸습니다.
12 수종이는 어디야? 	사무실	11
13 은경이는 정원으로 복귀했습니다.
14 은경이는 침실로 갔습니다.
15 경임이는 어디야? 	화장실	8
1 경임이는 사무실로 가버렸습니다.
2 경임이는 화장실로 이동했습니다.
3 경임이는 어디야? 	화장실	2
4 필웅이는 침실로 이동했습니다.
5 수종이는 복도로 갔습니다.


## bAbI 데이터 설명 
- 기계 이해 모델 중 하나인 **Memory Network** task에 적합한 데이터셋 
- 모델이 텍스트 내용에 대한 추론을 할 수 있도록 구성된 데이터셋
---
- 정보 문장이 연속해서 나온다. 
- 질문과 답으로 구성되어있는 문장이 나온다. 
    - 질문, 정답, 정답의 근거가 되는 문장의 index
- 새로운 스토리(정보 문장)이 시작되면 맨 앞 id number는 다시 1로 초기화된다.

## 1. 데이터 전처리 
### 기본 전처리
- **스토리, 질문, 답변**으로 분리해 저장하는 과정 
- supporting fact(정답의 근거가 되는 문장의 Index)는 저장하지 않는다.

In [6]:
def read_data(dir):
    stories, questions, answers = [], [], []
    story_temp = []  # 현재 시점의 스토리 
    lines = open(dir, 'rb')
    
    for line in lines:
        line = line.decode('utf-8')
        line = line.strip() # '\n' 제거
        idx, text = line.split(' ', 1)  # split(separator, maxsplit) : 1번 문자열 나눠 저장
        
        if int(idx) == 1: # 스토리별 분리 
            story_temp = [] # 초기화
        
        if '\t' in text: # 해당 줄이 질문과 답변이 있는 경우
            question, answer, _ = text.split('\t')
            stories.append([x for x in story_temp if x]) # 누적 스토리 저장 
            questions.append(question)
            answers.append(answer)
        
        else: 
            story_temp.append(text)
    
    lines.close()
    return stories, questions, answers

In [8]:
train_stories, train_questions, train_answers = read_data(train_path)
test_stories, test_questions, test_answers = read_data(test_path)

In [9]:
print("train 스토리 개수:", len(train_stories))
print("train 질문 개수:", len(train_questions))
print("train 답변 개수:", len(train_answers))
print("test 스토리 개수:", len(test_stories))
print("test 질문 개수:", len(test_questions))
print("test 답변 개수:", len(test_answers))

train 스토리 개수: 10000
train 질문 개수: 10000
train 답변 개수: 10000
test 스토리 개수: 1000
test 질문 개수: 1000
test 답변 개수: 1000


In [11]:
train_stories[0]

['필웅이는 화장실로 갔습니다.', '은경이는 복도로 이동했습니다.']

In [12]:
train_questions[:2]

['필웅이는 어디야? ', '수종이는 어디야? ']

In [13]:
train_answers[:2]

['화장실', '복도']

## 2. 데이터 토큰화
- customized konlpy를 사용해 토큰화한다.

In [15]:
from ckonlpy.tag import Twitter

twitter = Twitter()
twitter.add_dictionary('은경이', 'Noun')
test_morph = twitter.morphs('은경이는 사무실로 갔습니다.')

In [18]:
test_morph, type(test_morph)

(['은경이', '는', '사무실', '로', '갔습니다', '.'], list)

In [19]:
def get_token(sentence):
    twitter = Twitter()
    return twitter.morphs(sentence)

- 함수 테스트

In [22]:
train_stories_token_test = [get_token(sen) for sen in train_stories[0]]

In [24]:
train_stories_token_test

[['필웅이', '는', '화장실', '로', '갔습니다', '.'],
 ['은', '경이', '는', '복도', '로', '이동', '했습니다', '.']]

- 토큰 변환

In [26]:
train_stories_token = [get_token(sen) for sen in train_stories]
# train_questions_token = [get_token(sen) for sen in train_questions]
# train_answers_token = [get_token(sen) for sen in train_answers]
# test_stories_token = [get_token(sen) for sen in test_stories]
# test_questions_token = [get_token(sen) for sen in test_questions]
# test_answers_token = [get_token(sen) for sen in test_answers]

AttributeError: 'list' object has no attribute 'split'

# 루브릭 평가
---
1. **한국어의 특성에 알맞게 전처리가 진행되었다.**   

    - 한국어 특성에 따른 토큰화, 임베딩을 거쳐 데이터셋이 적절히 구성되었다.


2. **메모리 네트워크가 정상적으로 구현되어 학습이 안정적으로 진행되었다.**

    - validation loss가 안정적으로 수렴하는 것을 확인하고 이를 시각화하였다.


3. **메모리 네트워크를 통해 한국어 bAbI 태스크의 높은 정확도를 달성하였다.** 
 
     - 추론 태스크의 테스트 정확도가 90% 이상 달성하였다.


    
# 회고
---
## 어려웠던 부분 
- 

## 알아낸 점 혹은 모호한 부분 
- 

## 느낀 점 
- 